In [2]:
import numpy as np
import pandas as pd

In [3]:
dados= pd.read_csv('https://raw.githubusercontent.com/joaovictordds/Score_de_Credito/main/german.csv', sep=';')
dados.head(1)

,Creditability,Account_Balance,Duration_of_Credit_monthly,Payment_Status_of_Previous_Credit,Purpose,Credit_Amount,Value_Savings_Stocks,Length_of_current_employment,Instalment_per_cent,Sex_Marital_Status,...,Duration_in_Current_address,Most_valuable_available_asset,Age_years,Concurrent_Credits,Type_of_apartment,No_of_Credits_at_this_Bank,Occupation,No_of_dependents,Telephone,Foreign_Worker
0,1,1,18,4,2,1049,1,2,4,2,...,4,2,21,3,1,1,3,1,1,1


In [4]:
# Selecionando algumas variaveis para treino/testes
df = dados[['Creditability', 'Credit_Amount', 'Duration_of_Credit_monthly','Account_Balance','Length_of_current_employment','Sex_Marital_Status','Age_years', 'No_of_Credits_at_this_Bank']]
df.head(1)

,Creditability,Credit_Amount,Duration_of_Credit_monthly,Account_Balance,Length_of_current_employment,Sex_Marital_Status,Age_years,No_of_Credits_at_this_Bank
0,1,1049,18,1,2,2,21,1


In [17]:
from sklearn.model_selection import KFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LinearRegression
from sklearn.compose import ColumnTransformer
import joblib

# Seus dados e variáveis
X = df.drop('Credit_Amount', axis=1)
y = df['Credit_Amount']

# Defina as colunas categóricas e numéricas (ajuste conforme o seu conjunto de dados)
categorical_cols = ['Creditability', 'Account_Balance', 'Length_of_current_employment', 'Sex_Marital_Status', 'No_of_Credits_at_this_Bank']  # Lista das colunas categóricas
numeric_cols = ['Duration_of_Credit_monthly', 'Age_years']  # Lista das colunas numéricas

# Crie o ColumnTransformer para aplicar transformações diferentes às colunas categóricas e numéricas
preprocessor = ColumnTransformer(transformers=[('num', StandardScaler(), numeric_cols),  # Normaliza as colunas numéricas
                                               ('cat', OneHotEncoder(), categorical_cols)])  # Aplica OneHotEncoder nas colunas categóricas

# Crie o pipeline com as etapas desejadas
modelo = Pipeline([('preprocessor', preprocessor), ('regressor', LinearRegression())])

# Especifique o número de folds para a validação cruzada (ex: 5 folds)
kf = KFold(n_splits=5, shuffle=True, random_state=42)

# Realize a validação cruzada e calcule a média do erro quadrático médio negativo
neg_mse_scores = cross_val_score(modelo, X, y, cv=kf, scoring='neg_mean_squared_error')

# Transforme os scores negativos para positivos
mse_scores = -neg_mse_scores

# Exiba a média do erro quadrático médio
print("Média do Erro Quadrático Médio:", np.mean(mse_scores).round(4))

# Ajuste o modelo ao conjunto de dados completo
modelo.fit(X, y)

# Salve o modelo em um arquivo .pkl
joblib.dump(modelo, 'modelo_regressao.pkl')

Média do Erro Quadrático Médio: 4730844.5051


['modelo_regressao.pkl']

In [18]:
# Criando um novo DataFrame para novos dados
nvds = pd.DataFrame(columns=['Creditability','Account_Balance','Duration_of_Credit_monthly', 'Length_of_current_employment', 'Sex_Marital_Status', 'Age_years', 'No_of_Credits_at_this_Bank'])
nvds.loc[0, 'Creditability'] = 1
nvds.loc[0, 'Account_Balance'] = 2
nvds.loc[0, 'Duration_of_Credit_monthly'] = 36
nvds.loc[0, 'Length_of_current_employment'] = 1
nvds.loc[0, 'Sex_Marital_Status'] = 1
nvds.loc[0, 'Age_years'] = 19
nvds.loc[0, 'No_of_Credits_at_this_Bank'] = 1
#nvds.astype(float)

In [19]:
previsao_novos_dados = modelo.predict(nvds)
valor = previsao_novos_dados.astype(float)
print(f'Sugestão de valor: $ {valor.round(2)}')

Sugestão de valor: $ [6687.24]
